## Sobre o Projeto - Dados IMDB

O dataset utilizado é uma parte reduzida de uma junção de datasets de filmes e ratings dos mesmos. Ele contém várias colunas que trazem dados importantes sobre cada filme, e, a partir delas analisaremos o que podemos extrair de informação a partir desses registros.

Importaremos as bibliotecas necessárias e criaremos um dataframe a partir do .csv que está na pasta '/data'. Em seguida, para fazer uma análise mais completa, usaremos algumas funções básicas para encontrar particularidades de cada coluna.

In [11]:
# bibliotecas necessarias para a parte de transformação e limpeza do projeto
import pandas as pd
import math
from unidecode import unidecode

In [12]:
# importando os dados
df = pd.read_csv("data/imdb_data.csv", delimiter=";")
display(df)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.0,233.0
1,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",6.1,152.0
2,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",5.5,1965.0
3,tt0000015,short,Autour d'une cabine,Autour d'une cabine,0,1894,\N,2,"Animation,Short",6.2,947.0
4,tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",4.6,292.0
...,...,...,...,...,...,...,...,...,...,...,...
1995,tt7804296,tvEpisode,28th December 2017 Late News,28th December 2017 Late News,0,2017,\N,\N,News,NaN,NaN
1996,tt7804310,tvEpisode,Eitelkeiten,Eitelkeiten,0,1985,\N,\N,Family,NaN,NaN
1997,tt7804320,tvEpisode,Episode #1.7,Episode #1.7,0,2017,\N,\N,Drama,NaN,NaN
1998,tt7804336,video,Sex Machine,Sex Machine,1,1995,\N,60,Adult,2.7,6.0


In [13]:
# checando se existem dados nulos em alguma coluna
df.isnull().sum()

tconst               0
titleType            0
primaryTitle         0
originalTitle        0
isAdult              0
startYear            0
endYear              0
runtimeMinutes       0
genres               0
averageRating     1674
numVotes          1674
dtype: int64

Para melhorar a análise e a visualização do df, iremos fazer alguns ajustes no dataframe:

In [14]:
# mudando o nome da coluna tconst para id
df = df.rename(columns={'tconst': 'id'})

In [15]:
# checando se existem dados duplicados dentro do dataframe a partir da coluna id
duplicate = df[df.duplicated(subset=['id'])]

if duplicate.empty:
    print("Não há dados duplicados na coluna 'id'.")
else:
    print("Existem dados duplicados na coluna 'id':")
    print(duplicate)

Não há dados duplicados na coluna 'id'.


In [16]:
# checando os tipos de dado de cada coluna no df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   object 
 1   titleType       2000 non-null   object 
 2   primaryTitle    2000 non-null   object 
 3   originalTitle   2000 non-null   object 
 4   isAdult         2000 non-null   int64  
 5   startYear       2000 non-null   object 
 6   endYear         2000 non-null   object 
 7   runtimeMinutes  2000 non-null   object 
 8   genres          2000 non-null   object 
 9   averageRating   326 non-null    float64
 10  numVotes        326 non-null    float64
dtypes: float64(2), int64(1), object(8)
memory usage: 172.0+ KB


Podemos ver que a coluna startYear, endYear e runtimeMinutes estão como string, mas, para conseguir fazer uma análise melhor com esses dados, precisamos mudá-las para int e/ou float. Mas, podemos ver no display do df que essas colunas também vem com dados "nulos", identificados como '/N' ou '\\N'. Iremos tratar esses dados também.

Para esse dataframe, a melhor opção será deixar os dados como NaN, para que essas alterações não alterem os resultados estatisticos.

Unindo esses problemas, uma das melhores opções para alinhar esses dados será fazer uma função que trata tudo isso apenas passando-a na coluna especifica:

In [17]:
def clean_data_column(df, 
                      column):  
    df[f"{column}"] = df[f"{column}"].replace({"/N": None, "\\N": None})
    df[f"{column}"] = pd.to_numeric(df[f"{column}"], errors='coerce')

clean_data_column(df, 'startYear')
clean_data_column(df, 'endYear')
clean_data_column(df, 'runtimeMinutes')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   object 
 1   titleType       2000 non-null   object 
 2   primaryTitle    2000 non-null   object 
 3   originalTitle   2000 non-null   object 
 4   isAdult         2000 non-null   int64  
 5   startYear       1684 non-null   float64
 6   endYear         24 non-null     float64
 7   runtimeMinutes  587 non-null    float64
 8   genres          2000 non-null   object 
 9   averageRating   326 non-null    float64
 10  numVotes        326 non-null    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 172.0+ KB


Outro ponto que pode vir a ser um problema seria a despadronização dos caracteres nos campos string. Para isso, utilizaremos a biblioteca unidecode para ajustar todos esses valores.

In [19]:
def unidecode_upper(df, column):
    # função para padronizar strings a partir do unidecode e transformando-as em maiúsculo com o método .upper()
    df[f'{column}'] = df[f'{column}'].apply(lambda x: unidecode(x).upper())
    df[f"{column}"] = df[f"{column}"].replace({"/N": None, "\\N": None})

unidecode_upper(df, 'titleType')
unidecode_upper(df, 'primaryTitle')
unidecode_upper(df, 'originalTitle')
unidecode_upper(df, 'genres')

Agora, podemos ver como o dataframe está melhor para análise, utilizando display() e .describe() para ver algumas informações das colunas numéricas:

In [20]:
display(df)

,id,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000002,SHORT,LE CLOWN ET SES CHIENS,LE CLOWN ET SES CHIENS,0,1892.0,NaN,5.0,"ANIMATION,SHORT",6.0,233.0
1,tt0000004,SHORT,UN BON BOCK,UN BON BOCK,0,1892.0,NaN,12.0,"ANIMATION,SHORT",6.1,152.0
2,tt0000008,SHORT,EDISON KINETOSCOPIC RECORD OF A SNEEZE,EDISON KINETOSCOPIC RECORD OF A SNEEZE,0,1894.0,NaN,1.0,"DOCUMENTARY,SHORT",5.5,1965.0
3,tt0000015,SHORT,AUTOUR D'UNE CABINE,AUTOUR D'UNE CABINE,0,1894.0,NaN,2.0,"ANIMATION,SHORT",6.2,947.0
4,tt0000017,SHORT,ITALIENISCHER BAUERNTANZ,ITALIENISCHER BAUERNTANZ,0,1895.0,NaN,1.0,"DOCUMENTARY,SHORT",4.6,292.0
...,...,...,...,...,...,...,...,...,...,...,...
1995,tt7804296,TVEPISODE,28TH DECEMBER 2017 LATE NEWS,28TH DECEMBER 2017 LATE NEWS,0,2017.0,NaN,NaN,NEWS,NaN,NaN
1996,tt7804310,TVEPISODE,EITELKEITEN,EITELKEITEN,0,1985.0,NaN,NaN,FAMILY,NaN,NaN
1997,tt7804320,TVEPISODE,EPISODE #1.7,EPISODE #1.7,0,2017.0,NaN,NaN,DRAMA,NaN,NaN
1998,tt7804336,VIDEO,SEX MACHINE,SEX MACHINE,1,1995.0,NaN,60.0,ADULT,2.7,6.0


In [21]:
df.describe()

,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
count,2000.000000,1684.000000,24.000000,587.000000,326.000000,326.000000
mean,0.048500,1992.809976,2008.666667,31.872232,6.477301,210.917178
std,0.214874,37.993473,13.222730,36.252419,1.382653,1123.662888
min,0.000000,1892.000000,1972.000000,1.000000,2.700000,5.000000
25%,0.000000,1990.000000,2005.000000,10.000000,5.500000,13.000000
50%,0.000000,2010.000000,2014.500000,21.000000,6.450000,27.000000
75%,0.000000,2016.000000,2017.250000,43.000000,7.500000,105.500000
max,1.000000,2021.000000,2020.000000,302.000000,9.800000,18983.000000


In [22]:
df.isnull().sum()

id                   0
titleType            0
primaryTitle         0
originalTitle        0
isAdult              0
startYear          316
endYear           1976
runtimeMinutes    1413
genres             136
averageRating     1674
numVotes          1674
dtype: int64

Como dito anteriormente, no caso desse dataframe é necessário deixarmos os dados nulos para não interferir na análise dos dados numéricos. Porém, podemos ver que os dados em formato string não retornam nulo, o que quer dizer que todos os registros tem pelo menos informações importantes como id, titleType, primaryTitle, entre outras.

Com o dataframe limpo e pronto, iremos exportar o arquivo em formato .csv e analisá-lo em outro notebook.

In [23]:
df.to_csv('processed_data/imdb_processed_data.csv', index=False, sep=';')